In [1]:
import pandas as pd
import smtplib
import os
import json
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email import encoders

In [8]:
# Configuración de comportamiento
ORIGEN = 'locuestkey@gmail.com'
ASUNTO = 'Correo de prueba'
SERVIDOR_SMTP = 'smtp.gmail.com:587'
CORREO_TEST = True

COLUMNA_CORREO = 'CORREO_DOCENTE'
COLUMNAS = [
    'CODIGO_CURSO',
    'NOMBRE_TUTOR',
    'CORREO_TUTOR',
    'TELEFONO_TUTOR',
    'PROFESIONAL+',
    'BACHILLER',
    'TÉCNICO',
    'DESCONOCIDO',
    'TOTAL',
    'INDICE_HOMOGENEIDAD'
]

In [3]:
MSG = """Bucaramanga, 17 de abril de 2022
Respetado profesional
{formador}
Programa Misión TIC 2022

En vista a que va a iniciar el Ciclo 1, adjunto a este correo se encuentran las listas de clase de cada uno de los grupos a los que usted pertenece y con la información de la caracterización de cada curso.
Esta caracterización está dada por el nivel académico de cada tripulante, donde está dividido en 4 clases: Bachiller, Técnico, Profesional+ y Desconocido. 
A partir de estas 4 posibles clases, se calcula un índice de homogeneidad por cada grupo. Este índice varía de 0 hasta 1, a más cercano esté el índice a 1 significa que el grupo es más homogéneo.
Esta información se le proporciona para que tenga una idea general sobre los tripulantes de sus cursos, con el fin de abordar, de ser necesario, diferentes estrategias pedagógicas.

ESTE EMAIL SE ENVIÓ DE FORMA AUTOMÁTICA, NO RESPONDA A ESTE CORREO. Cualquier información, contacte a misiontic.monitor@uis.edu.co o rectoria.misiontic@uis.edu.co.
"""

In [4]:
ruta = './INPUT/INFO_CURSOS/LISTA_CURSOS_FORMADORES_EMAIL.xlsx' if not CORREO_TEST else './INPUT/INFO_CURSOS/TEST.xlsx'

In [5]:
f = open('CREDENCIALES.json')
CREDENCIALES = json.load(f)
f.close()
ruta = './INPUT/INFO_CURSOS/LISTA_CURSOS_FORMADORES_EMAIL.xlsx' if not CORREO_TEST else './INPUT/INFO_CURSOS/TEST.xlsx'
datos = pd.read_excel(ruta, engine = 'openpyxl')

In [6]:
def enviarCorreo(origen, destino, asunto, mensaje, adjuntos = []):
    multipart = MIMEMultipart()
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart.attach(MIMEText(mensaje))
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

In [9]:
destinos = datos[COLUMNA_CORREO].unique()
servidor = smtplib.SMTP(SERVIDOR_SMTP)
servidor.starttls()
servidor.login(CREDENCIALES['USUARIO'], CREDENCIALES['PASS'])
for destino in destinos:
    dfD = datos[datos[COLUMNA_CORREO] == destino]
    nombreFormador = dfD['NOMBRE_DOCENTE'].iloc[0]
    adjuntos = []
    for index, row in dfD.iterrows():
        ruta = './INPUT/INFO_CURSOS/LISTAS/' + row['CODIGO_CURSO'] + '.xlsx'
        adjuntos.append(ruta)
    nombreArchivo = 'INFO_GRUPOS_' + nombreFormador.replace(' ', '_')
    ruta = './Temp/' + nombreArchivo + '.xlsx'
    dfD[COLUMNAS].to_excel(ruta, index = False)
    adjuntos.append(ruta)
    msg = enviarCorreo(ORIGEN, destino, ASUNTO, MSG.format(formador = nombreFormador), adjuntos)
    servidor.sendmail(ORIGEN, destino, msg.as_string())
servidor.quit()

(221,
 b'2.0.0 closing connection c131-20020a379a89000000b0069c903625absm5526037qke.102 - gsmtp')

In [ ]:
# Limpiado de archivos temporales
